## Talos-POC Demo (Notebook)

- 输入 1 条用户消息
- Graph 运行到 `interrupt()` 时自动批准（`approval=True`）
- 打印 `__interrupt__` payload 与最终 `final_state`

> 依赖外部 LLM/环境变量时，请提前确认本机环境已配置好（比如 `.env`）。



In [ ]:
from __future__ import annotations

import uuid
from typing import Any

from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.types import Command

from src.main import talos_agent
from src.utils.tools import _pretty


def run_once(
    *,
    conversation: list[AnyMessage],
    thread_id: str,
    auto_approve: bool = True,
    comment: str = "",
) -> dict[str, Any]:
    config: Any = {"configurable": {"thread_id": thread_id}}

    next_input: Any = {
        "user_input": list(conversation),
        "messages": list(conversation),
    }

    while True:
        interrupt_payload: Any | None = None
        last_state: dict[str, Any] | None = None

        for state in talos_agent.stream(next_input, config=config, stream_mode="values"):
            last_state = state
            if "__interrupt__" in state:
                interrupt_payload = state["__interrupt__"]
                break

        if interrupt_payload is None:
            return last_state or {}

        print("\n--- HUMAN REVIEW REQUIRED ---")
        print(_pretty(interrupt_payload))

        if not auto_approve:
            raise RuntimeError("Graph requested human approval; set auto_approve=True to continue in this demo.")

        next_input = Command(resume={"approval": True, "comment": comment})


def run_demo(user_text: str) -> dict[str, Any]:
    thread_id = f"notebook-{uuid.uuid4()}"
    conversation: list[AnyMessage] = [HumanMessage(content=user_text)]

    final_state = run_once(conversation=conversation, thread_id=thread_id, auto_approve=True)

    print("\n--- FINAL STATE ---")
    print(_pretty(final_state))

    return final_state



In [ ]:
sample = "我正在进行水杨酸 (Salicylic Acid) 的 乙酰化反应 制备乙酰水杨酸 (Aspirin)。需要进行中控监测 (IPC)"

_ = run_demo(sample)